#**SQLAlchemy - Parte II**

- Introdução ao SQL Alchemy - Parte II
  - Compreendendo melhor relacionamentos
  - SQL Alchemy assíncrono
  - Refatorando o código
  - Validando os códigos assíncronos



##**Relacionamentos**
Muitas vezes, a criação de um banco de dados que dá suporte a uma aplicação exige a criação de mais de uma tabela.

O projeto de distribuição dos dados em bancos de dados relacionais tenta maximizar a efiência do armazenamento no que diz respeito à performance, mas também tenta evitar redundâncias e inconsistências. O estabelecimento de relacionamentos entre diversas tabelas ajuda nisso.

###Um exemplo: 
Considere que você precisa gerenciar os dados de pacientes de uma clínica médica (por exemplo: CPF, nome e e-mail).

Em um primeiro momento, tudo que você precisaria seria colocar estes dados em uma tabela **Paciente**. Todos os pacientes pentencem à clínica e estão devidamente representados no banco de dados, certo?

Imagine agora que a aplicação precise controlar os as consultas que cada paciente fez com cada mṕedico da clínica.

Além dos dados cadastrais de cada médico (CRM, nome, especialidade), seria preciso registrar dados de cada consulta (data e hora da consulta, o médico que atendeu a consulta e o paciente atendido).

Isso significa que precisamos criar duas tabelas novas (**Medico** e **Consulta**) e estabelecer relacionamentos entre todas as tabelas do banco de dados para manter os dados sob controle.

###Cardinalidade em Relacionamentos
1. Relacionamento "Um para Um"
  - Cada elemento de uma tabela A está relacionado a um elemento em outra tabela B
  - Exemplo: Cada médico possui apenas uma especialidade
    - 1 médico <-> 1 especialidade
2. Relacionamento "Um para Muitos"
  - Cada elemento de uma tabela A está relacionado a vários elementos em outra tabela B
  - Exemplo: Cada médico atende diversas consultas
    - 1 médico <-> N consultas
3. Relacionamento "Muitos para Muitos"
  - Cada elemento de uma tabela A está relacionado a vários elementos em outra tabela B, mas isso também se aplica no sentido inverso (um elemento de B pode estar relacionado a diversos elementos de A)
  - Exemplo: Cada médico atende diversas consultas e cada paciente pode se consultar com diversos médicos
    - médico -> consulta <- paciente

Estes relacionamentos podem ser implementados utilizando-se o conceito de **Chave Estrangeira** (**Foreign Key**).




###**Relacionamentos no SQLAlchemy**
Para estabelecer um relação entre duas tabelas usando o SQLAlchemy, a definição de chave estrangeira é feita na Classe python que representa a tabela do banco de dados.


###**Exemplo Usando SQLite**


####**Criando o Banco de Dados da Clínica**

```python
import sqlalchemy
# engine
engine = sqlalchemy.create_engine('sqlite:///clinica.db',echo=False)
# mapeamento ORM
from sqlalchemy.orm import declarative_base
Base = declarative_base()
# importando elementos
from sqlalchemy import Column, String, Integer, ForeignKey

# tabela de Médicos
class Medico(Base):
    __tablename__ = 'tab_medico'
    CRM = Column(Integer, primary_key=True)
    nome = Column(String(20))
    especialidade = Column(String(20))

# tabela de Pacientes
class Paciente(Base):
    __tablename__ = 'tab_paciente'
    CPF = Column(Integer, primary_key=True)
    nome = Column(String(20))
    email = Column(String(20))

# tabela de Consultas
class Consulta(Base):
    __tablename__ = 'tab_consulta'
    id = Column(Integer, primary_key=True)
    data_hora = Column(String) # SQLite não entende formato datetime
    CRM_medico = Column(Integer,ForeignKey("tab_medico.CRM"))
    CPF_paciente = Column(Integer,ForeignKey("tab_paciente.CPF"))

Base.metadata.create_all(engine)
```


####**Inserindo Dados no Banco de Dados da Clínica**
```python
import sqlalchemy
# criando engine
engine = sqlalchemy.create_engine('sqlite:///clinica.db',echo=False)
# importando classes
from pgm1_criar_BD import Medico,Paciente,Consulta

# criando uma sessão no banco de dados
from sqlalchemy.orm import sessionmaker
SessaoBD = sessionmaker(bind=engine) # cria uma classe "Sessao"
sessao = SessaoBD() # intancia um objeto do tipo "SessaoBD" chamado "sessao"

# instanciando objetos
med1 = Medico(CRM=111,nome='Carlos',especialidade='Clínica Geral')
med2 = Medico(CRM=222,nome='Sofia',especialidade='Cardiologia')
med3 = Medico(CRM=333,nome='Paulo',especialidade='Pediatria')

pac1 = Paciente(CPF=12312312300,nome='Maria',email='maria@gmail.com')
pac2 = Paciente(CPF=45645645600,nome='Pedro',email='pedro@yahoo.com')
pac3 = Paciente(CPF=78978978900,nome='Ana',email='ana@microsoft.com')

con1 = Consulta(id=1,data_hora='2022/09/12-08:30', CRM_medico=med1.CRM, CPF_paciente=pac1.CPF)
con2 = Consulta(id=2,data_hora='2022/09/12-16:00', CRM_medico=med2.CRM, CPF_paciente=pac2.CPF)
con3 = Consulta(id=3,data_hora='2022/09/12-11:00', CRM_medico=med3.CRM, CPF_paciente=pac3.CPF)
con4 = Consulta(id=4,data_hora='2022/09/15-08:30', CRM_medico=med2.CRM, CPF_paciente=pac1.CPF)
con5 = Consulta(id=5,data_hora='2022/09/15-10:00', CRM_medico=med2.CRM, CPF_paciente=pac2.CPF)
con6 = Consulta(id=6,data_hora='2022/09/16-09:00', CRM_medico=med3.CRM, CPF_paciente=99999999999)
con7 = Consulta(id=7,data_hora='2022/09/16-09:00', CRM_medico=444, CPF_paciente=pac3.CPF)

# inserindo dados de médicos
sessao.add_all([med1,med2,med3])
sessao.commit()

# inserindo dados de pacientes
sessao.add_all([pac1,pac2,pac3])
sessao.commit()

# inserindo dados de consultas

# -----Para funcionar FKs no SQLite--------
sessao.execute('PRAGMA foreign_keys = ON;')
# -----------------------------------------
sessao.add_all([con1,con2,con3,con4,con5]) # este comando funciona (obedece FKs)
sessao.commit()

# -----Para funcionar FKs no SQLite--------
sessao.execute('PRAGMA foreign_keys = ON;')
# -----------------------------------------
sessao.add_all([con6,con7]) # este comando não funciona (não obedece FKs)
sessao.commit()
```

####**Consultando Dados no Banco de Dados da Clínica**
```python
import sqlalchemy
# criando engine
engine = sqlalchemy.create_engine('sqlite:///clinica.db',echo=False)
# importando classes
from pgm1_criar_BD import Medico,Paciente,Consulta
# criando uma sessão no banco de dados
from sqlalchemy.orm import sessionmaker
SessaoBD = sessionmaker(bind=engine) # cria uma classe "Sessao"
sessao = SessaoBD() # intancia um objeto do tipo "SessaoBD" chamado "sessao"

# medico --------------------------------
resposta = sessao.query(Medico) # retorna uma lista de médicos
for resp in resposta:
    print('Médico:',resp.CRM, resp.nome)

# paciente --------------------------------
resposta = sessao.query(Paciente) # retorna uma lista de pacientes
for resp in resposta:
    print('Paciente:',resp.CPF, resp.nome)

# consultas --------------------------------
resposta = sessao.query(Consulta) # retorna uma lista de consultas
for resp in resposta:
    print('Consulta:',resp.id, resp.data_hora, resp.CRM_medico, resp.CPF_paciente)
```

####**Consultando Dados em Mais de Uma Tabela (JOIN)**
```python
import sqlalchemy
from sqlalchemy import and_, or_
# criando engine
engine = sqlalchemy.create_engine('sqlite:///clinica.db',echo=False)
# importando classes
from pgm1_criar_BD import Medico,Paciente,Consulta
# criando uma sessão no banco de dados
from sqlalchemy.orm import sessionmaker
SessaoBD = sessionmaker(bind=engine) # cria uma classe "Sessao"
sessao = SessaoBD() # intancia um objeto do tipo "SessaoBD" chamado "sessao"

# medico x consulta --------------------------------
print('\nMEDICO X CONSULTA')
resposta = sessao.query(Medico,Consulta). \
                  filter(and_(Medico.nome=='Sofia', \
                              Medico.CRM == Consulta.CRM_medico)).all()
for med,con in resposta:
    print('Médico:',med.CRM, med.nome, 'Consulta:', con.data_hora, con.CPF_paciente)

# paciente x consulta -----------------------------
print('\nPACIENTE X CONSULTA')
resposta = sessao.query(Paciente,Consulta). \
                  filter(and_(Paciente.CPF==12312312300, \
                              Paciente.CPF == Consulta.CPF_paciente)).all()
for pac,con in resposta:
    print('Paciente:',pac.nome, 'Consulta:', con.data_hora, con.CRM_medico)

# paciente x consulta x médico --------------------------
print('\nPACIENTE X CONSULTA X MÉDICO')
resposta = sessao.query(Paciente,Consulta,Medico). \
                  filter(and_(Paciente.CPF == Consulta.CPF_paciente, \
                              Medico.CRM == Consulta.CRM_medico)).all()
for pac,con,med in resposta:
    print('Paciente:',pac.nome, 'Consulta:', con.data_hora, 'Médico:', med.nome)
```

###**Vamos Praticar**
Você foi encarregado de criar uma aplicação para controlar os fornecedores e produtos que uma loja de materias de escritório utiliza.

Seu banco de dados deve armazenar:
- Informações sobre os fornecedores
  - Id, nome, e-mail, fone
- Informações sobre os produtos
  - Id, descrição e quem é o fornecedor do produto

Sua aplicação deve ser capaz de:
- Incluir e excluir um fornecedor
- Incluir e excluir um produto
- Exibir um relatório contendo a descrição de cada produto e seu respectivo fornecedor


###**Desafio**
O cliente gostou da sua solução para o problema anterior, mas pediu que fosse criada uma nova versão da aplicação onde um produto puidesse ter vários fornecedores.

Ou seja: o relacionamento entre produtos e fornacedores, que antes era de "um para muitos" (1 fornecedor fornece N produtos) agora passou a ser "muitos para muitos" ( N fornecedor fornecem N produtos).

O cliente também solicitou que o aplicativo tivesse os seguintes relatórios:
- Para um produto informado, exibir todos os seus fornecedores
- Para um fornecedor informado, exibir todos os produtos que ele fornece
